# **Iterative Methods for Sparse Linear Systems (From Scratch)**

**Author:** Kaj Hansteen Izora  

This project showcases several iterative methods for solving large, sparse linear systems of the form $A x = b$. All core routines—steepest descent, conjugate gradient, and preconditioned conjugate gradient—are implemented from the ground up, giving a clear illustration of how these algorithms work under the hood. Additionally, the code demonstrates how to handle sparse matrices efficiently using CSR (Compressed Sparse Row) format, along with how to incorporate different preconditioners (Jacobi and symmetric Gauss-Seidel).

---

## **1. Overview of Key Algorithms**

1. **Steepest Descent**  
   - Two variants (following Algorithm 1 and Algorithm 2 from standard references/notes).  
   - Uses the negative gradient of the residual to iteratively improve the solution.  
   - Simple to implement but can be relatively slow to converge.

2. **Conjugate Gradient (CG)**  
   - A classic method for positive-definite systems that significantly outperforms plain steepest descent.  
   - Iterates in directions that are conjugate with respect to the matrix $A$, speeding up convergence.

3. **Preconditioned Conjugate Gradient (PCG)**  
   - Applies a preconditioner $M$ to reduce the condition number of $A$.  
   - We use both **Jacobi** and **symmetric Gauss-Seidel** preconditioners.  
   - Typically converges faster than standard CG, albeit with a more expensive iteration step.

We also explore theoretical convergence estimates (related to the condition number) and compare them with observed iteration counts.

---

## **2. Working with Sparse CSR Matrices**

### **2.1. Imports**

In [25]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

### **2.2. Loading and CSR Helper Functions**

Below are utility functions that load a matrix from a file into CSR format, extract the number of rows, and perform a matrix-vector multiplication—all written from scratch (rather than using built-in methods).

In [26]:
def load_as_csr(filename):
    """
    Loads a sparse matrix from a custom file format into CSR representation.
    
    Parameters
    ----------
    filename : str
        Path to the file containing the matrix data.
        
    Returns
    -------
    (I, J, V) : tuple of np.ndarray
        CSR representation: I (index pointer), J (column indices), and V (nonzero values).
    """
    # Read matrix dimensions and number of nonzeros
    m, n, nnz = np.genfromtxt(filename, max_rows=1, dtype=None)
    # Read the data as a list of tuples [(row_i, col_j, value), ...]
    data = np.genfromtxt(filename, skip_header=1, dtype=None)

    # Allocate arrays
    I = np.zeros(m + 1, dtype=int)
    J = np.zeros(nnz, dtype=int)
    V = np.zeros(nnz)

    # Count nonzeros per row
    for k in range(nnz):
        i = data[k][0]
        I[i+1] += 1

    # Convert to cumulative sums
    for i in range(int(m)):
        I[i+1] += I[i]

    # Fill J and V based on row placements
    for k in range(nnz):
        i, j, val = data[k]
        idx = I[i]
        I[i] += 1
        J[idx] = j
        V[idx] = val

    # Shift I array back to correct pointers
    for i in reversed(range(int(m))):
        I[i+1] = I[i]
    I[0] = 0

    return (I, J, V)

def num_rows(A):
    """
    Returns the number of rows in the CSR matrix A = (I, J, V).
    """
    return len(A[0]) - 1

def matvec(A, v):
    """
    Performs the matrix-vector product for a CSR matrix A and vector v.
    """
    I, J, V = A
    m = num_rows(A)
    w = np.zeros(m)
    for i in range(m):
        begin = I[i]
        end = I[i+1]
        for idx in range(begin, end):
            j = J[idx]
            w[i] += V[idx] * v[j]
    return w

---

## **3. Steepest Descent**

Steepest descent is often taught as the most direct gradient-based approach to solving $Ax = b$. We show two implementations:

- **`steepest_descent_1`** follows a classical textbook/lecture style of computing the residual and step size each iteration.
- **`steepest_descent_2`** stores one less matrix-vector product per iteration.

### **3.1. Implementation**

In [27]:
def steepest_descent_1(A, b, tol, max_it=2000):
    """
    Implements classical steepest descent (Algorithm 1 style).
    Converges when the norm of the residual is below `tol`.
    """
    n = len(b)
    x = np.zeros(n)
    it = 0
    while True:
        r = b - matvec(A, x)
        alpha = np.dot(r, r) / np.dot(r, matvec(A, r))
        x = x + alpha * r
        it += 1
        
        if np.linalg.norm(r) < tol:
            print(f"Converged in {it} iterations")
            break
        if it >= max_it:
            print("Max iterations Reached")
            break
    return x, it

def steepest_descent_2(A, b, tol, max_it=2000):
    """
    Steepest descent with a slight optimization (Algorithm 2 style).
    Stores `z = A r` to avoid re-computing it in the alpha step.
    """
    n = len(b)
    x = np.zeros(n)
    r = b - matvec(A, x)
    z = np.zeros(n)
    alpha = 0
    it = 0
    while True:
        r = r - alpha * z
        z = matvec(A, r)
        alpha = np.dot(r, r) / np.dot(r, z)
        x = x + alpha * r
        it += 1
        
        if np.linalg.norm(r) < tol:
            print(f"Converged in {it} iterations")
            break
        if it >= max_it:
            print("Max iterations Reached")
            break
    return x, it

### **3.2. Testing Steepest Descent**

We load four example matrices (`16.txt`, `25.txt`, `50.txt`, `64.txt`) which vary in size and condition numbers. Each matrix is solved against a randomly generated right-hand side.

In [28]:
files = ["16.txt", "25.txt", "50.txt", "64.txt"]
tolerance = 1e-8

for file in files:
    A_csr_tuple = load_as_csr(file)
    n = int(file.split('.')[0])
    b = np.random.default_rng(0).random(n)  # random RHS
    # Solve using both versions
    x1, iterations1 = steepest_descent_1(A_csr_tuple, b, tolerance)
    x2, iterations2 = steepest_descent_2(A_csr_tuple, b, tolerance)

    print(f"\nFile: {file}")
    print(f"Steepest Descent 1 iterations: {iterations1}")
    print(f"Steepest Descent 2 iterations: {iterations2}")

    # Compare with exact solution from spsolve
    I, J, V = A_csr_tuple
    A_csr = csr_matrix((V, J, I), shape=(len(I)-1, max(J)+1))
    x_exact = spsolve(A_csr, b)

    # Basic correctness check
    # (direct float comparison can be tricky, so in practice, one might compare norms)
    print("\nactual answer", x_exact)
    # Simple check
    if np.allclose(x1, x_exact, rtol=1e-6, atol=1e-10):
        print("SD1 is correct")
    if np.allclose(x2, x_exact, rtol=1e-6, atol=1e-10):
        print("SD2 is correct")
    print("-" * 60)

Converged in 607 iterations
Converged in 607 iterations

File: 16.txt
Steepest Descent 1 iterations: 607
Steepest Descent 2 iterations: 607

actual answer [0.53198622 0.09899997 0.68102428 0.00172617 0.55390107 0.87347616
 0.46273783 0.9043583  0.83372745 0.19049786 0.66853579 0.16824415
 0.22832795 0.53371259 0.25781822 0.4518783 ]
SD1 is correct
SD2 is correct
------------------------------------------------------------
Converged in 837 iterations
Converged in 837 iterations

File: 25.txt
Steepest Descent 1 iterations: 837
Steepest Descent 2 iterations: 837

actual answer [ 5.16246348e+01 -5.72735133e+00 -1.41688466e+01  3.36685402e+00
  1.30856991e+01  2.35415246e+00  3.95129133e+00  3.11327406e+00
 -6.63028231e+00  5.25829866e+00  5.40795979e+00 -7.92076305e-01
  2.09922311e+01 -5.72575914e-02  2.15755518e+01 -9.94338413e-01
  5.61752076e+00  7.90741857e+00  5.37835719e+00  1.22606610e+01
 -8.65993019e+00  4.56114180e+00  3.94504901e+01  1.36704408e+00
  6.15144214e+01]
SD1 is corr

**Reflections on Steepest Descent**  
- `steepest_descent_1` performs 2 matrix-vector products per iteration (`matvec(A, x)` and `matvec(A, r)`).  
- `steepest_descent_2` reuses the vector $z = Ar$, saving a bit of computation each iteration.  
- In practice, `steepest_descent_2` often converges in the same number of steps but with fewer total flops.

---

## **4. Convergence Estimates**

A well-known theoretical bound says that steepest descent requires approximately  
$$
\frac{\kappa(A) - 1}{\kappa(A) + 1} \quad\text{and}\quad
\text{(a related formula for CG involves } \sqrt{\kappa(A)}\text{)},
$$  
where $\kappa(A)$ is the condition number of $A$. We can estimate the number of iterations by rearranging that ratio into a tolerance formula.

### **4.1. Condition Number and Estimates**

In [29]:
def condition_number(A):
    """
    Approximates the condition number of the matrix in A (CSR format)
    using the largest and smallest eigenvalues (via eigsh).
    """
    A_sp = sp.csr_matrix((A[2], A[1], A[0]))
    lmin, _ = sp.linalg.eigsh(A_sp, k=1, which="SM")
    lmax, _ = sp.linalg.eigsh(A_sp, k=1, which="LM")
    return lmax[0]/lmin[0]

def estimate_iterations(condition_num, tol):
    r = (condition_num - 1) / (condition_num + 1)
    return np.ceil(np.log(tol) / np.log(r))

files = ["16.txt", "25.txt", "50.txt", "64.txt"]
tol = 1e-8

for file in files:
    A_tuple = load_as_csr(file)
    cond_num = condition_number(A_tuple)
    its_needed = estimate_iterations(cond_num, tol)
    print(f"File: {file}, Condition Number: {cond_num}, Estimated Iterations: {its_needed}")

File: 16.txt, Condition Number: 126.85801089570502, Estimated Iterations: 1169.0
File: 25.txt, Condition Number: 100.00000000000485, Estimated Iterations: 922.0
File: 50.txt, Condition Number: 100.00000000000337, Estimated Iterations: 922.0
File: 64.txt, Condition Number: 170.84087457719286, Estimated Iterations: 1574.0


Observe how these estimates relate to the actual iteration counts from steepest descent above. Typically, the estimates are upper bounds or close approximations—actual iteration counts might be lower.

---

## **5. Conjugate Gradient (CG)**

Next, we implement the **Conjugate Gradient** method for symmetric positive-definite (SPD) systems. This method converges much faster than steepest descent in many cases because it chooses directions that minimize the error in a conjugate subspace.

### **5.1. Implementation**

In [30]:
def conjugate_gradient(A, b, tol):
    """
    Conjugate Gradient (CG) implementation from scratch.

    Parameters
    ----------
    A : tuple (I, J, V) in CSR format
    b : np.ndarray
        Right-hand side vector
    tol : float
        Convergence tolerance
    
    Returns
    -------
    x : np.ndarray
        Approximate solution to A x = b
    it : int
        Number of iterations performed
    """
    max_it = 2000
    n = len(b)
    x = np.zeros(n)
    r = b - matvec(A, x)
    d = r.copy()
    
    z_dot_z = np.dot(r, r)
    it = 0

    while True:
        Ad = matvec(A, d)
        alpha = z_dot_z / np.dot(d, Ad)
        x += alpha * d
        r -= alpha * Ad
        r_dot_r = np.dot(r, r)
        beta = r_dot_r / z_dot_z
        d = r + beta * d
        z_dot_z = r_dot_r
        it += 1

        if np.linalg.norm(r) < tol:
            print(f"Converged in {it} iterations")
            break
        if it >= max_it:
            print("Max iterations reached")
            break
    return x, it

### **5.2. Testing CG**

In [31]:
tolerance = 1e-8
files = ["16.txt", "25.txt", "50.txt", "64.txt"]

for file in files:
    A_tuple = load_as_csr(file)
    n = int(file.split('.')[0])
    b = np.random.default_rng(0).random(n)
    x_cg, its_cg = conjugate_gradient(A_tuple, b, tolerance)
    
    print(f"\nFile: {file}")
    print(f"CG iterations: {its_cg}")
    
    # Compare with the spsolve solution
    I, J, V = A_tuple
    A_csr_mat = csr_matrix((V, J, I), shape=(len(I)-1, max(J)+1))
    x_true = spsolve(A_csr_mat, b)
    
    print("\nactual answer", x_true)
    if np.allclose(x_cg, x_true, rtol=1e-6, atol=1e-10):
        print("CG is correct")
    print("-" * 60)

Converged in 17 iterations

File: 16.txt
CG iterations: 17

actual answer [0.53198622 0.09899997 0.68102428 0.00172617 0.55390107 0.87347616
 0.46273783 0.9043583  0.83372745 0.19049786 0.66853579 0.16824415
 0.22832795 0.53371259 0.25781822 0.4518783 ]
CG is correct
------------------------------------------------------------
Converged in 31 iterations

File: 25.txt
CG iterations: 31

actual answer [ 5.16246348e+01 -5.72735133e+00 -1.41688466e+01  3.36685402e+00
  1.30856991e+01  2.35415246e+00  3.95129133e+00  3.11327406e+00
 -6.63028231e+00  5.25829866e+00  5.40795979e+00 -7.92076305e-01
  2.09922311e+01 -5.72575914e-02  2.15755518e+01 -9.94338413e-01
  5.61752076e+00  7.90741857e+00  5.37835719e+00  1.22606610e+01
 -8.65993019e+00  4.56114180e+00  3.94504901e+01  1.36704408e+00
  6.15144214e+01]
CG is correct
------------------------------------------------------------
Converged in 59 iterations

File: 50.txt
CG iterations: 59

actual answer [-2.8080864   9.23668557 28.29912267 18.

### **5.3. CG Convergence Estimates**

Recall that CG typically satisfies  
$$
\|x^{(k)} - x^*\|_A \le 2 \left(\frac{\sqrt{\kappa(A)} - 1}{\sqrt{\kappa(A)} + 1}\right)^k \|x^{(0)} - x^*\|_A.
$$

We can estimate the required number of iterations similarly:

In [32]:
def estimate_iterations(kappa, delta):
    """
    Approximates how many CG iterations are needed for
    a given condition number kappa and tolerance delta.
    """
    r = (np.sqrt(kappa) - 1) / (np.sqrt(kappa) + 1)
    return np.ceil(np.log(delta/2) / np.log(r))

files = ["16.txt", "25.txt", "50.txt", "64.txt"]
delta = 1e-8

for file in files:
    A_tuple = load_as_csr(file)
    kappa = condition_number(A_tuple)
    it_est = estimate_iterations(kappa, delta)
    print(f"File: {file}, Condition Number: {kappa}, Estimated CG Iterations: {it_est}")

File: 16.txt, Condition Number: 126.85801089570502, Estimated CG Iterations: 108.0
File: 25.txt, Condition Number: 100.00000000000544, Estimated CG Iterations: 96.0
File: 50.txt, Condition Number: 100.00000000000333, Estimated CG Iterations: 96.0
File: 64.txt, Condition Number: 170.8408745771942, Estimated CG Iterations: 125.0


We can then compare these theoretical estimates with the actual iteration counts from the CG tests above.

---

## **6. Preconditioning**

### **6.1. Jacobi and Symmetric Gauss-Seidel**

Preconditioning modifies the system $A x = b$ to $M^{-1} A x = M^{-1} b$, where $M$ is easier to invert. Below we implement two standard preconditioners:

- **Jacobi**: Uses the diagonal of $A$.  
- **Symmetric Gauss-Seidel**: Uses the lower- and upper-triangular factors for forward/backward substitution.

#### **6.1.1. Extract Diagonal & Jacobi**

In [33]:
def get_diag(A):
    """
    Returns the diagonal entries of A (CSR) as a 1D numpy array.
    """
    I, J, V = A
    n = num_rows(A)
    D = np.zeros(n)
    for i in range(n):
        row_start = I[i]
        row_end = I[i+1]
        for idx in range(row_start, row_end):
            col_idx = J[idx]
            if col_idx == i:
                D[i] = V[idx]
                break
    return D

def jacobi_preconditioner(A):
    """
    Creates a function M(x) = D^{-1} x for the Jacobi preconditioner.
    """
    D = get_diag(A)
    def jacobi(x):
        return x / D
    return jacobi

#### **6.1.2. Symmetric Gauss-Seidel**

We need forward and backward solvers for the lower and upper triangular parts of $A$. These were built from scratch previously.

In [34]:
def tril_solve(A, b):
    """
    Forward solve for lower-triangular part of A in CSR.
    """
    I, J, V = A
    x = np.zeros(len(b))
    n = num_rows(A)
    for i in range(n):
        x[i] = b[i]
        row_start = I[i]
        row_end = I[i+1]
        diag_val = 1
        for k in range(row_start, row_end):
            j = J[k]
            if j < i:
                x[i] -= V[k] * x[j]
            elif j == i:
                diag_val = V[k]
        x[i] /= diag_val
    return x

def triu_solve(A, b):
    """
    Backward solve for upper-triangular part of A in CSR.
    """
    I, J, V = A
    x = np.zeros(len(b))
    n = num_rows(A)
    for i in reversed(range(n)):
        x[i] = b[i]
        row_start = I[i]
        row_end = I[i+1]
        diag_val = 1
        for k in range(row_start, row_end):
            j = J[k]
            if j > i:
                x[i] -= V[k] * x[j]
            elif j == i:
                diag_val = V[k]
        x[i] /= diag_val
    return x

def symmetric_gauss_seidel_preconditioner(A):
    """
    Creates a function M(x) corresponding to Symmetric Gauss-Seidel: y = L^{-1} D U^{-1} x.
    """
    def sgs(x):
        Uinv_x = triu_solve(A, x)
        D = get_diag(A)
        D_Uinv_x = D * Uinv_x
        return tril_solve(A, D_Uinv_x)
    return sgs

---

### **6.2. Preconditioned Conjugate Gradient (PCG)**

Finally, we implement a **Preconditioned CG** routine, closely mirroring the standard CG algorithm but applying $M$ to residual vectors:

In [35]:
def preconditioned_conjugate_gradient(A, M, b, tol):
    """
    Preconditioned Conjugate Gradient method (Algorithm 2 style).
    
    Parameters
    ----------
    A : tuple
        CSR matrix representation of A.
    M : function
        Preconditioner function that computes M(x).
    b : np.ndarray
        Right-hand side vector
    tol : float
        Tolerance
    """
    max_it = 2000
    it = 0
    n = len(b)
    x = np.zeros(n)

    s = b - matvec(A, x)  # residual
    r = M(s)             # M^-1 * s
    d = r.copy()
    Ad = matvec(A, d)

    s_dot_r = np.dot(s, r)
    
    while True:
        alpha = s_dot_r / np.dot(d, Ad)
        x += alpha * d
        z = s.copy()
        s -= alpha * Ad
        Mz = r
        r = M(s)
        s_dot_r_new = np.dot(s, r)
        beta = s_dot_r_new / np.dot(z, Mz)
        d = r + beta * d
        Ad = matvec(A, d)
        s_dot_r = s_dot_r_new

        it += 1
        if np.linalg.norm(r) < tol:
            print(f"PCG converged in {it} iterations")
            break
        if it >= max_it:
            print("Max iterations reached (PCG)")
            break
    return x, it + 1

Note that each iteration uses one `matvec(A, d)` call and one application of the preconditioner $M$. This is typically referred to as the “optimized” form of PCG.

---

## **7. Experiments with Preconditioning**

We test each matrix with **Symmetric Gauss-Seidel** (SGS) vs. **Jacobi** as the preconditioner:

In [36]:
tolerance = 1e-8
files = ["16.txt", "25.txt", "50.txt", "64.txt"]

for file in files:
    A_tuple = load_as_csr(file)
    n = int(file.split('.')[0])
    b = np.random.default_rng(0).random(n)

    # Symmetric Gauss-Seidel
    M_sgs = symmetric_gauss_seidel_preconditioner(A_tuple)
    x_sgs, its_sgs = preconditioned_conjugate_gradient(A_tuple, M_sgs, b, tolerance)

    # Jacobi
    M_jacobi = jacobi_preconditioner(A_tuple)
    x_jacobi, its_jacobi = preconditioned_conjugate_gradient(A_tuple, M_jacobi, b, tolerance)

    print(f"\nFile: {file}")
    print(f"Results with Symmetric Gauss-Seidel => Iterations: {its_sgs}")
    print(f"Results with Jacobi => Iterations: {its_jacobi}")

    # Compare with exact solution
    I, J, V = A_tuple
    A_csr_test = csr_matrix((V, J, I), shape=(len(I)-1, max(J)+1))
    x_exact = spsolve(A_csr_test, b)
    print("\nactual answer", x_exact)

    # Quick correctness checks
    if np.allclose(x_sgs, x_exact, rtol=1e-6, atol=1e-10):
        print("SGS is correct")
    if np.allclose(x_jacobi, x_exact, rtol=1e-6, atol=1e-10):
        print("Jacobi is correct")
    print("-" * 60)

PCG converged in 9 iterations
PCG converged in 15 iterations

File: 16.txt
Results with Symmetric Gauss-Seidel => Iterations: 10
Results with Jacobi => Iterations: 16

actual answer [0.53198622 0.09899997 0.68102428 0.00172617 0.55390107 0.87347616
 0.46273783 0.9043583  0.83372745 0.19049786 0.66853579 0.16824415
 0.22832795 0.53371259 0.25781822 0.4518783 ]
SGS is correct
Jacobi is correct
------------------------------------------------------------
PCG converged in 13 iterations
PCG converged in 24 iterations

File: 25.txt
Results with Symmetric Gauss-Seidel => Iterations: 14
Results with Jacobi => Iterations: 25

actual answer [ 5.16246348e+01 -5.72735133e+00 -1.41688466e+01  3.36685402e+00
  1.30856991e+01  2.35415246e+00  3.95129133e+00  3.11327406e+00
 -6.63028231e+00  5.25829866e+00  5.40795979e+00 -7.92076305e-01
  2.09922311e+01 -5.72575914e-02  2.15755518e+01 -9.94338413e-01
  5.61752076e+00  7.90741857e+00  5.37835719e+00  1.22606610e+01
 -8.65993019e+00  4.56114180e+00  3.

---

## **8. Reflections and Trade-Offs**

1. **Comparison of Methods**  
   - **Steepest Descent**: Easiest to implement but generally slow convergence, highly sensitive to condition number.  
   - **Conjugate Gradient**: Substantially faster for symmetric positive-definite matrices, leveraging conjugate directions.  
   - **Preconditioned CG**: Can drastically reduce iteration counts when a good $M$ is chosen, especially for ill-conditioned problems.

2. **Preconditioner Trade-Offs**  
   - **Jacobi** is simple (just diagonal inversion) but often provides modest improvement.  
   - **Symmetric Gauss-Seidel** typically converges faster but requires more work (forward and backward passes).  
   - A more complex preconditioner can mean fewer iterations but higher cost per iteration.

3. **Practical Considerations**  
   - The best choice depends on matrix structure. If $A$ is close to diagonal, Jacobi might be enough. If $A$ is more dense or has more complex coupling, a stronger preconditioner (like SGS or incomplete factorization) might be preferred.  
   - Real-world usage frequently employs specialized preconditioners tuned to the physics or geometry behind $A$.

Through these implementations, we see how “from scratch” methods can reveal deeper insights into the interplay between condition numbers, iteration counts, and overall computational cost.

---

# **Conclusion**

In this project, we constructed a complete pipeline for tackling sparse linear systems. The process begins with loading and handling matrices in CSR format, followed by two variations of the Steepest Descent method as a baseline for iterative solves. We then explored the Conjugate Gradient method, which is especially effective for symmetric positive-definite systems, and extended it by incorporating Jacobi and Symmetric Gauss-Seidel preconditioners to form a Preconditioned Conjugate Gradient approach. We verified the accuracy of these implementations by comparing solutions against SciPy’s `spsolve` and examined their convergence patterns in light of theoretical estimates tied to the condition number. The results indicate that more sophisticated methods, particularly Conjugate Gradient and its preconditioned variant, converge significantly faster than Steepest Descent, with strong preconditioners yielding further gains in efficiency. We welcome any questions and encourage suggestions for future enhancements, such as incomplete LU or incomplete Cholesky preconditioners, as well as potential applications to real-world PDE problems.
